In [10]:
import json
import pandas as pd

In [11]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)

In [12]:
# make discord name
discord_username_full = user['username'] + '#' + str(user['discriminator'])

In [13]:
fact_messages = pd.read_csv('fact_messages.csv')
dim_channels = pd.read_csv('dim_channel.csv')

In [14]:
fact_messages.head()

,message_id,timestamp,contents,attachment_link,channel_id
0,950964769018494986,2022-03-09 03:54:29.005000+00:00,I graduated at the peak of covid too,NaN,950796898736934953
1,950964735648616469,2022-03-09 03:54:21.049000+00:00,haha couldnt find anything in the UK,NaN,950796898736934953
2,950964482467831818,2022-03-09 03:53:20.686000+00:00,yeah alot of malaysians in manchester,NaN,950796898736934953
3,950964002421342308,2022-03-09 03:51:26.234000+00:00,yeah no optical,NaN,950796898736934953
4,950963949950611496,2022-03-09 03:51:13.724000+00:00,Manchester,NaN,950796898736934953


In [15]:
dim_channels.head()

,channel_id,channel_name,channel_type_key,server_id
0,950796898736934953,Direct Message with yellowpika#2074,1,NaN
1,913728489910321163,Direct Message with vlad101vlad#3325,1,NaN
2,964109025807568916,Direct Message with namf#5582,1,NaN
3,882368940058230794,Direct Message with nemo24#5701,1,NaN
4,819752706762997791,NaN,0,NaN


In [16]:
dim_channels[dim_channels['server_id'] == 946913009211703168]

,channel_id,channel_name,channel_type_key,server_id
7,946982427853738085,mods-team,0,9.469130e+17
12,950754264974753852,Tax,11,9.469130e+17
13,962605177989386250,UK,11,9.469130e+17
15,950435368061251595,makan,0,9.469130e+17
16,947495222613516298,emotes-suggestions,0,9.469130e+17
...,...,...,...,...
551,946969444591210537,creative,0,9.469130e+17
554,961663453255319582,bookclub,0,9.469130e+17
556,989196605456539728,archived-ticket-0170,0,9.469130e+17
557,961833434085466112,Salary submission form improvements,11,9.469130e+17


In [17]:
fact_messages[fact_messages['channel_id'] = ]

SyntaxError: invalid syntax (3617882968.py, line 1)